In [1]:
import dask
import dask.bag as db
import dask.dataframe as dd
from dask.distributed import get_worker
from dask.distributed import Client, progress

In [2]:
# client = Client(processes=True)
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB', processes=True)
client

Client Scheduler: tcp://127.0.0.1:36109 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [3]:
# client.close()

In [4]:
import glob
import spacy
import pandas as pd

from wb_nlp import dir_manager

In [5]:
f = [f for f in glob.glob(dir_manager.get_path_from_root('src', 'wb_nlp', 'cleaning', '*.py'))]

In [6]:
def read_file(f):
    worker = get_worker()

    try:
        nlp = worker.nlp
    except AttributeError:
        nlp = spacy.load('en_core_web_sm')
        worker.nlp = nlp

    content = {}
    with open(f) as fl:
        doc = nlp(fl.read())
        content[f] = [token.lemma_ for token in doc]
        with open(f + '.txt', 'w') as ff:
            fl.seek(0)
            ff.write(fl.read())

    return content

In [13]:
b = db.from_sequence(f * 100, npartitions=4)
file_reader = b.map(read_file)

In [8]:
%%time
res = file_reader.compute()

CPU times: user 1.93 s, sys: 1.14 s, total: 3.07 s
Wall time: 18.4 s


In [9]:
len(res)

30

In [12]:
%%time
res = file_reader.compute()

CPU times: user 718 ms, sys: 446 ms, total: 1.16 s
Wall time: 5.85 s


In [14]:
%%time
res = file_reader.compute()

CPU times: user 5.37 s, sys: 3.42 s, total: 8.79 s
Wall time: 50.7 s
